In [1]:
from bs4 import BeautifulSoup
import requests

# About this project

- Scrape price information from Google Finance, given some ticker and exchange.
- Summarize portfolio valuation for an arbitrary number of positions.
- Solution should reflect USD amounts only, but also support positions listed in other currencies i.e. should support some sort of FX-ing capability.

In [43]:
from dataclasses import dataclass
from tabulate import tabulate

ticker = "GOOG"
exchange = "NASDAQ"
url = f"https://www.google.com/finance/quote/{ticker}:{exchange}"

@dataclass
class Stock:
    ticker: str
    exchange: str
    price: float = 0
    currency: str = "USD"
    usd_price: float = 0

    def __post_init__(self):
        price_info = get_price_information(self.ticker, self.exchange)

        if price_info['ticker'] == self.ticker:
            self.price = price_info['price']
            self.currency = price_info['currency']
            self.usd_price = price_info['usd_price']

@dataclass
class Position:
    stock: Stock
    quantity: int
    
@dataclass
class Portfolio:
    positions: list[Position]
    
    def get_total_value(self):
        total_value = 0

        for position in self.positions:
            total_value += position.quantity * position.stock.usd_price
        return total_value

def get_price_information(ticker, exchange):
    # Define the target url
    url = f"https://www.google.com/finance/quote/{ticker}:{exchange}"
    # Make a request and get response from server url
    response = requests.get(url)
    # Create BeautifulSoup object
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the element target, the element that contains data
    element_target = soup.find('div', attrs={'data-last-price': True})
    # Get the price data
    price = float(element_target.attrs.get('data-last-price'))
    # Get the currency data
    currency = element_target.attrs.get('data-currency-code')
    # Define the usd price, assume that the currency is USD
    usd_price = price
    # If the currency not usd
    if currency.lower() != 'usd':
        # Get the rate currency
        rate = get_fx_to_usd(currency)
        # Update the usd price
        usd_price = rate * price
        
        
    return {'ticker': ticker, 
            'exchange': exchange, 
            'price': price, 
            'currency': currency, 
            'usd_price': usd_price}

def get_fx_to_usd(currency):
    # Define the target url
    url = f"https://www.google.com/finance/quote/{currency}-USD"
    # Make a request and get response from server url
    response = requests.get(url)
    # Create BeautifulSoup object
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the element target, the element that contains data
    element_target = soup.find('div', attrs={'data-last-price': True})
    # Get the rate current currency to usd
    rate = float(element_target.attrs.get('data-last-price'))
    return rate
    


def display_portfolio_summary(portfolio):
    if not isinstance(portfolio, Portfolio):
        raise TypeError("Please provide an instance of the Portfolio type")

    portfolio_value = portfolio.get_total_value()

    position_data = []

    for position in portfolio.positions:
        position_data.append([
            position.stock.ticker,
            position.stock.exchange,
            position.quantity,
            position.stock.usd_price,
            position.quantity * position.stock.usd_price,
            position.quantity * position.stock.usd_price / portfolio_value * 100
        ])

    print(tabulate(position_data,
                   headers=['Ticker', 'Exchange', 'Quantity', 'Price', 'Market Value', '% Allocation'],
                   tablefmt='psql',
                   floatfmt='.2f',
                  ))
    print(f"Total portfolio value: {portfolio_value:,.2f}")


# Run program
shop = Stock("SHOP", "TSE")
msft = Stock("MSFT", "NASDAQ")
googl = Stock("GOOGL", "NASDAQ")

portfolio = Portfolio([Position(shop, 10), Position(msft, 2), Position(googl, 30)])

display_portfolio_summary(portfolio)

+----------+------------+------------+---------+----------------+----------------+
| Ticker   | Exchange   |   Quantity |   Price |   Market Value |   % Allocation |
|----------+------------+------------+---------+----------------+----------------|
| SHOP     | TSE        |         10 |   89.84 |         898.39 |          12.57 |
| MSFT     | NASDAQ     |          2 |  418.01 |         836.02 |          11.70 |
| GOOGL    | NASDAQ     |         30 |  180.35 |        5410.50 |          75.73 |
+----------+------------+------------+---------+----------------+----------------+
Total portfolio value: 7,144.91
